In [4]:
import os
import numpy as np
import imageio
from utils.utils import printProgressBar
from utils.image_transform import pil_loader

def data_crop(inputPath, outputPath):
    imgList = [f for f in os.listdir(
        inputPath) if os.path.splitext(f)[1] == ".jpg"]
    cx, cy = 89, 121
    nImgs = len(imgList)
    if not os.path.isdir(outputPath):
        os.mkdir(outputPath)
    for index, item in enumerate(imgList):
        printProgressBar(index, nImgs)
        path = os.path.join(inputPath, item)
        img = np.array(pil_loader(path))
        img = img[cy - 64: cy + 64, cx - 64: cx + 64]
        path = os.path.join(outputPath, item)
        imageio.imwrite(path, img)
    printProgressBar(nImgs, nImgs)

dataset_path = '../dataset/celeba'
output_dataset = '../datasets'

data_crop(dataset_path, output_dataset)

Cropping dataset...
 |####################################################################################################| 100.0% 


In [1]:
import torch 
import data_processing.DataLoader as dl
import yaml
from torch.backends import cudnn
from pro_gan_pytorch.PRO_GAN import ProGAN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True
torch.manual_seed(3)

In [6]:
def get_config(conf_file):
    from easydict import EasyDict as edict
    with open(conf_file, "r") as file_descriptor:
        data = yaml.full_load(file_descriptor)
    return edict(data)

In [7]:
def train():
    config = get_config('./train.config')
    Dataset = dl.FlatDirectoryImageDataset
    dataset = Dataset(
        data_dir=config.images_dir,
        transform=dl.get_transform(config.img_dims)
    )

    pro_gan = ProGAN(
        depth=config.depth,
        latent_size=config.latent_size,
        learning_rate=config.learning_rate,
        beta_1=config.beta_1,
        beta_2=config.beta_2,
        eps=config.eps,
        drift=config.drift,
        n_critic=config.n_critic,
        use_eql=config.use_eql,
        loss=config.loss_function,
        use_ema=config.use_ema,
        ema_decay=config.ema_decay,
        device=device)
    
    pro_gan.train(
        dataset=dataset,
        epochs=config.epochs,
        fade_in_percentage=config.fade_in_percentage,
        start_depth=0,
        batch_sizes=config.batch_sizes,
        num_workers=config.num_workers,
        feedback_factor=config.feedback_factor,
        log_dir=config.log_dir,
        sample_dir=config.sample_dir,
        num_samples=config.num_samples,
        checkpoint_factor=config.checkpoint_factor,
        save_dir=config.save_dir,)

In [8]:
train()

Current Configuration: {'images_dir': '../datasets', 'folder_distributed': False, 'log_dir': 'training_runs/celebA/losses/', 'sample_dir': 'training_runs/celebA/generated_samples/', 'save_dir': 'training_runs/celebA/saved_models/', 'img_dims': [128, 128], 'use_eql': True, 'depth': 6, 'latent_size': 512, 'learning_rate': 0.001, 'beta_1': 0, 'beta_2': 0.99, 'eps': 1e-08, 'drift': 0.001, 'n_critic': 1, 'use_ema': True, 'ema_decay': 0.999, 'epochs': [4, 8, 8, 8, 8, 8], 'fade_in_percentage': [50, 50, 50, 50, 50, 50], 'batch_sizes': [512, 256, 128, 64, 64, 64], 'loss_function': 'wgan-gp', 'num_samples': 36, 'num_workers': 3, 'feedback_factor': 100, 'checkpoint_factor': 2}
total examples in training:  202599
Generator_configuration:
DataParallel(
  (module): Generator(
    (initial_block): GenInitialBlock(
      (conv_1): _equalized_deconv2d(512, 512, 4, 4)
      (conv_2): _equalized_conv2d(512, 512, 3, 3)
      (pixNorm): PixelwiseNorm()
      (lrelu): LeakyReLU(negative_slope=0.2)
    )
   

Elapsed: [0:01:05.561807]  batch: 195  d_loss: -1.316710  g_loss: -0.067293
Elapsed: [0:01:06.222407]  batch: 198  d_loss: -1.243473  g_loss: -0.339419
Elapsed: [0:01:06.888013]  batch: 201  d_loss: -1.299180  g_loss: -0.141193
Elapsed: [0:01:07.547613]  batch: 204  d_loss: -1.292654  g_loss: -0.225742
Elapsed: [0:01:08.175183]  batch: 207  d_loss: -1.231853  g_loss: -0.206830
Elapsed: [0:01:08.865084]  batch: 210  d_loss: -1.194180  g_loss: -0.359840
Elapsed: [0:01:09.532084]  batch: 213  d_loss: -1.181226  g_loss: -0.193993
Elapsed: [0:01:10.190683]  batch: 216  d_loss: -1.133746  g_loss: -0.451123
Elapsed: [0:01:10.802523]  batch: 219  d_loss: -1.135074  g_loss: -0.246988
Elapsed: [0:01:11.442104]  batch: 222  d_loss: -1.171340  g_loss: -0.451226
Elapsed: [0:01:12.052660]  batch: 225  d_loss: -1.087731  g_loss: -0.363837
Elapsed: [0:01:12.653205]  batch: 228  d_loss: -1.126681  g_loss: -0.500357
Elapsed: [0:01:13.266117]  batch: 231  d_loss: -1.069517  g_loss: -0.512119
Elapsed: [0:

Elapsed: [0:02:35.115176]  batch: 120  d_loss: -0.525234  g_loss: -0.631086
Elapsed: [0:02:35.735740]  batch: 123  d_loss: -0.545554  g_loss: -0.753664
Elapsed: [0:02:36.352303]  batch: 126  d_loss: -0.562907  g_loss: -0.413079
Elapsed: [0:02:36.967364]  batch: 129  d_loss: -0.505463  g_loss: -0.630975
Elapsed: [0:02:37.571913]  batch: 132  d_loss: -0.542762  g_loss: -0.456128
Elapsed: [0:02:38.181467]  batch: 135  d_loss: -0.463453  g_loss: -0.431765
Elapsed: [0:02:38.792022]  batch: 138  d_loss: -0.530552  g_loss: -0.496868
Elapsed: [0:02:39.399575]  batch: 141  d_loss: -0.514410  g_loss: -0.455581
Elapsed: [0:02:40.018137]  batch: 144  d_loss: -0.482265  g_loss: -0.541047
Elapsed: [0:02:40.636587]  batch: 147  d_loss: -0.517824  g_loss: -0.285516
Elapsed: [0:02:41.247143]  batch: 150  d_loss: -0.508776  g_loss: -0.643803
Elapsed: [0:02:41.861701]  batch: 153  d_loss: -0.498381  g_loss: -0.562917
Elapsed: [0:02:42.485013]  batch: 156  d_loss: -0.517490  g_loss: -0.281795
Elapsed: [0:

Elapsed: [0:04:03.225678]  batch: 45  d_loss: -0.383101  g_loss: 0.353871
Elapsed: [0:04:03.871265]  batch: 48  d_loss: -0.403177  g_loss: 0.266017
Elapsed: [0:04:04.511354]  batch: 51  d_loss: -0.373104  g_loss: 0.150258
Elapsed: [0:04:05.160273]  batch: 54  d_loss: -0.410424  g_loss: 0.098372
Elapsed: [0:04:05.769827]  batch: 57  d_loss: -0.356433  g_loss: 0.142525
Elapsed: [0:04:06.400189]  batch: 60  d_loss: -0.396613  g_loss: 0.102003
Elapsed: [0:04:07.024366]  batch: 63  d_loss: -0.388851  g_loss: -0.001096
Elapsed: [0:04:07.656941]  batch: 66  d_loss: -0.423978  g_loss: 0.188039
Elapsed: [0:04:08.270498]  batch: 69  d_loss: -0.375059  g_loss: 0.221989
Elapsed: [0:04:08.908079]  batch: 72  d_loss: -0.391789  g_loss: 0.309243
Elapsed: [0:04:09.526641]  batch: 75  d_loss: -0.410192  g_loss: 0.250350
Elapsed: [0:04:10.154212]  batch: 78  d_loss: -0.368480  g_loss: 0.171632
Elapsed: [0:04:10.764767]  batch: 81  d_loss: -0.369611  g_loss: 0.279691
Elapsed: [0:04:11.373321]  batch: 84 

Elapsed: [0:05:11.274052]  batch: 375  d_loss: -0.301373  g_loss: 0.317139
Elapsed: [0:05:11.899633]  batch: 378  d_loss: -0.327587  g_loss: 0.376902
Elapsed: [0:05:12.520198]  batch: 381  d_loss: -0.342606  g_loss: 0.300501
Elapsed: [0:05:13.141762]  batch: 384  d_loss: -0.341672  g_loss: 0.447600
Elapsed: [0:05:13.759325]  batch: 387  d_loss: -0.310119  g_loss: 0.226902
Elapsed: [0:05:14.384893]  batch: 390  d_loss: -0.353032  g_loss: 0.421342
Elapsed: [0:05:14.983438]  batch: 393  d_loss: -0.303078  g_loss: 0.276527
Elapsed: [0:05:15.530935]  batch: 396  d_loss: -0.335016  g_loss: 0.365486
Time taken for epoch: 103.567 secs

Epoch: 4
Elapsed: [0:05:37.485770]  batch: 1  d_loss: -0.344755  g_loss: 0.458923
Elapsed: [0:05:37.976217]  batch: 3  d_loss: -0.367698  g_loss: 0.321063
Elapsed: [0:05:38.603788]  batch: 6  d_loss: -0.336908  g_loss: 0.450382
Elapsed: [0:05:39.221349]  batch: 9  d_loss: -0.363932  g_loss: 0.381526
Elapsed: [0:05:39.833905]  batch: 12  d_loss: -0.333914  g_loss

...

Elapsed: [3:06:57.448560]  batch: 825  d_loss: -0.221117  g_loss: 0.945582
Elapsed: [3:07:07.610827]  batch: 840  d_loss: -0.356478  g_loss: 0.669412
Elapsed: [3:07:17.765090]  batch: 855  d_loss: -0.268307  g_loss: 0.157850
Elapsed: [3:07:27.922673]  batch: 870  d_loss: -0.184743  g_loss: 0.734282
Elapsed: [3:07:38.077018]  batch: 885  d_loss: -0.245947  g_loss: 1.141804
Elapsed: [3:07:48.231409]  batch: 900  d_loss: -0.163877  g_loss: 1.236658
Elapsed: [3:07:58.396517]  batch: 915  d_loss: -0.215576  g_loss: 0.427092
Elapsed: [3:08:08.555691]  batch: 930  d_loss: -0.026594  g_loss: 0.832562
Elapsed: [3:08:18.702858]  batch: 945  d_loss: -1.283988  g_loss: 4.075447
Elapsed: [3:08:28.855111]  batch: 960  d_loss: 0.004673  g_loss: 0.516878
Elapsed: [3:08:39.015374]  batch: 975  d_loss: -0.290549  g_loss: 0.442587
Elapsed: [3:08:49.163749]  batch: 990  d_loss: -0.453466  g_loss: 0.926559
Elapsed: [3:08:59.310678]  batch: 1005  d_loss: -0.336984  g_loss: 0.256623
Elapsed: [3:09:09.472940]

Elapsed: [3:25:40.558672]  batch: 870  d_loss: -0.177383  g_loss: 1.063550
Elapsed: [3:25:51.078317]  batch: 885  d_loss: -0.129874  g_loss: 0.770687
Elapsed: [3:26:01.607026]  batch: 900  d_loss: -0.099339  g_loss: 0.866841
Elapsed: [3:26:12.134008]  batch: 915  d_loss: -0.336523  g_loss: 0.614275
Elapsed: [3:26:22.470421]  batch: 930  d_loss: -0.105575  g_loss: 0.736462
Elapsed: [3:26:32.623673]  batch: 945  d_loss: 0.166757  g_loss: 0.190405
Elapsed: [3:26:42.781938]  batch: 960  d_loss: -0.112997  g_loss: 0.671176
Elapsed: [3:26:52.941148]  batch: 975  d_loss: -0.179693  g_loss: 1.195689
Elapsed: [3:27:03.101877]  batch: 990  d_loss: 0.104106  g_loss: 0.760126
Elapsed: [3:27:13.260138]  batch: 1005  d_loss: -0.330552  g_loss: 1.002034
Elapsed: [3:27:23.415390]  batch: 1020  d_loss: -0.152489  g_loss: 0.605369
Elapsed: [3:27:33.570022]  batch: 1035  d_loss: -0.239358  g_loss: 1.001952
Elapsed: [3:27:43.727358]  batch: 1050  d_loss: -0.224556  g_loss: 0.537782
Elapsed: [3:27:53.88055

Elapsed: [4:14:00.277902]  batch: 1860  d_loss: 0.501658  g_loss: -0.212219
Elapsed: [4:14:40.069214]  batch: 1891  d_loss: 0.556821  g_loss: 0.179510
Elapsed: [4:15:19.875478]  batch: 1922  d_loss: 0.530693  g_loss: -0.496636
Elapsed: [4:15:59.661496]  batch: 1953  d_loss: -0.357806  g_loss: 1.414618
Elapsed: [4:16:39.449274]  batch: 1984  d_loss: 0.249696  g_loss: 0.090565
Elapsed: [4:17:19.256515]  batch: 2015  d_loss: 0.523858  g_loss: -0.936297
Elapsed: [4:17:59.034920]  batch: 2046  d_loss: 0.240754  g_loss: 0.351568
Elapsed: [4:18:38.822170]  batch: 2077  d_loss: -1.071426  g_loss: 2.570646
Elapsed: [4:19:18.847895]  batch: 2108  d_loss: 0.313137  g_loss: 0.012628
Elapsed: [4:19:59.301003]  batch: 2139  d_loss: 0.075049  g_loss: 0.466188
Elapsed: [4:20:39.083520]  batch: 2170  d_loss: -0.851245  g_loss: -0.492248
Elapsed: [4:21:18.896114]  batch: 2201  d_loss: 0.109138  g_loss: -0.391283
Elapsed: [4:21:58.692880]  batch: 2232  d_loss: 0.145083  g_loss: 0.996989
Elapsed: [4:22:38

Elapsed: [5:31:53.683386]  batch: 2046  d_loss: -0.647226  g_loss: 0.604797
Elapsed: [5:32:41.978709]  batch: 2077  d_loss: -0.409062  g_loss: 0.574927
Elapsed: [5:33:30.181284]  batch: 2108  d_loss: 0.608196  g_loss: -2.685709
Elapsed: [5:34:16.362893]  batch: 2139  d_loss: -0.590494  g_loss: 0.201613
Elapsed: [5:35:05.390771]  batch: 2170  d_loss: -0.345277  g_loss: 0.409487
Elapsed: [5:35:52.985603]  batch: 2201  d_loss: -0.387188  g_loss: 1.035323
Elapsed: [5:36:35.773511]  batch: 2232  d_loss: -0.011294  g_loss: 1.402979
Elapsed: [5:37:19.480063]  batch: 2263  d_loss: -0.168626  g_loss: 1.709123
Elapsed: [5:38:02.964207]  batch: 2294  d_loss: 0.044659  g_loss: 1.641277
Elapsed: [5:38:47.418356]  batch: 2325  d_loss: -0.181139  g_loss: 1.833383
Elapsed: [5:39:36.122873]  batch: 2356  d_loss: -0.043256  g_loss: 1.016981
Elapsed: [5:40:28.777804]  batch: 2387  d_loss: -0.398891  g_loss: 1.285658
Elapsed: [5:41:20.552063]  batch: 2418  d_loss: -0.291125  g_loss: 1.990951
Elapsed: [5:4

KeyboardInterrupt: 